# Predict

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import joblib

In [27]:
ann_model = load_model('ann_model.keras')
rf_model = joblib.load('rf_model.pkl')
scaler = joblib.load('scaler.pkl')

In [3]:
flashaidtemp = pd.read_csv('flashaidtemp.csv')

In [4]:
X = flashaidtemp.drop(columns=['Claim Risk'])
y = flashaidtemp['Claim Risk']

### Compile Time Input

In [5]:
user_input = {
    'Frequency Months': 12,
    'Adult': 1,
    'Child': 0,
    'Sum Assured': 100000,
    'Insurance Price with GST': 41,
    'Max Age': 60,
    'Min Age': 18,
    'Insurance Type_Health': 0,
    'Insurance Type_Accidental Death': 1,
    'Insurance Type_Critical illness': 0,
    'Pre-existing disease waiting period years': 4,
    'Day Care Surgery waiting period years': 0,
    'Cooling Period days': 30,
    'Room Rent ICU Available': 0,
    'Room Rent Normal Percent': 0,
    'Room Rent ICU Percent': 0,
    'Cataract Available': 0,
    'Cataract waiting years': 0,
    'Maternity Available': 0,
    'Maternity Flat': 0,
    'Hospital Covered': 1,
    'Pre Hospitalization 30 days': 0,
    'Post Hospitalization 60 days': 0,
    'Advanced Treatment Available': 0,
    'Accident Available': 1,
    'Critical Available': 0,
    'HospiCash Available': 0
}

### Run Time Input

In [6]:
def get_user_input():
    user_input = {}
    for column in flashaidtemp.columns[0:10]:             
        if column == "Insurance Type_Health":
            value = int(input(f"Is Healthcare insurance available? Enter 1 for Yes and 0 for No: "))
            user_input[column] = value
            if value == 1:
                # Set other insurance types to 0 if healthcare is 1
                user_input["Insurance Type_Accidental Death"] = 0
                user_input["Insurance Type_Critical illness"] = 0
            else:
                value = int(input(f"Is Accidental Death insurance available? Enter 1 for Yes and 0 for No: "))
                user_input["Insurance Type_Accidental Death"] = value
                if value == 1:
                    user_input["Insurance Type_Critical illness"] = 0
                    user_input["Accident Available"] = 1
                    user_input["Critical Available"] = 0
                else:
                    value = int(input(f"Is Critical insurance available? Enter 1 for Yes and 0 for No: "))
                    user_input["Insurance Type_Critical illness"] = value
                    user_input["Accident Available"] = 0
                    user_input["Critical Available"] = 1
                    if value == 0:
                        print("Error, must be of one of 3 types")
                        exit(1)
                    
        if column not in user_input: 
            user_input[column] = float(input(f"Enter value for {column}: "))            
                    
    for column in flashaidtemp.columns[10:-1]:
            if user_input.get("Insurance Type_Health") == 0:
                    if column in ["Day Care Surgery waiting period years", "Advanced Treatment Available",
                                  "Room Rent Normal Percent", "Room Rent ICU Percent", "Maternity Available",
                                  "Room Rent ICU Available", "Maternity Flat", "Pre Hospitalization 30 days", 
                                  "Cataract Available", "Cataract waiting years", "Post Hospitalization 60 days", 
                                  "HospiCash Available"]:
                        user_input[column] = 0
                        continue
                                           
            if column not in user_input: 
                if column.endswith("Available"):
                    value = int(input(f"Is {column.replace(' Available', '')} available? Enter 1 for Yes and 0 for No: "))
                    user_input[column] = value
                
                elif column.endswith("0 days"):
                    value = int(input(f"Is {column} available? Enter 1 for Yes and 0 for No: "))
                    user_input[column] = value
                
                else:
                    value = float(input(f"Enter value for {column}: "))
                    user_input[column] = value
                
            if column == "Cataract Available" and value == 0:
                user_input["Cataract waiting years"] = 0
            if column == "Maternity Available" and value == 0:
                user_input["Maternity Flat"] = 0
            if column == "Room Rent ICU Available" and value == 0:
                user_input["Room Rent Normal Percent"] = 0
                user_input["Room Rent ICU Percent"] = 0
    
    return user_input

In [7]:
#user_input = get_user_input()

### Prediction

In [8]:
user_input_df = pd.DataFrame([user_input])
#user_input_df

In [28]:
# Align columns with the training data
user_input_aligned = user_input_df.reindex(columns=X.columns, fill_value=0)

# Scale the input
user_input_scaled = scaler.transform(user_input_aligned)

In [32]:
predicted_claim_ratio_ann = ann_model.predict(user_input_scaled)

1/1 [==============================] - 0s 16ms/step


In [11]:
predicted_claim_ratio_rf = rf_model.predict(user_input_aligned)

In [33]:
claim_risk_ann = predicted_claim_ratio_ann[0][0] * 100
claim_risk_rf = predicted_claim_ratio_rf[0] * 100

print(f"Predicted Claim Risk (ANN): {claim_risk_ann:.2f}%")
print(f"Predicted Claim Risk (Random Forest): {claim_risk_rf:.2f}%")

Predicted Claim Risk (ANN): 37.03%
Predicted Claim Risk (Random Forest): 36.30%
